In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

dataset=pd.read_csv('results.csv')

In [2]:
import mysql.connector

In [3]:
# Establish MYSQL connection
            
mydb= mysql.connector.connect(host= '127.0.0.1',
    user='root',
    password='######')

mydb.cursor()

In [4]:
#Database Creation

mycursor=mydb.cursor()
mycursor.execute("DROP DATABASE IF EXISTS football")
mycursor.execute("CREATE DATABASE IF NOT EXISTS football")

In [5]:
# Access the database

mydb=mysql.connector.connect(
    host= '127.0.0.1',
    user='root',
    password='######',
    database='football')



In [6]:
mycursor=mydb.cursor(buffered=True)

# we use buffered option as True to enable multiple execute operations. 

In [7]:
mycursor.execute("USE football")

In [8]:
mycursor.execute(" CREATE TABLE results (Match_date date, home_team VARCHAR(72), away_team VARCHAR(72), home_score INT, away_score INT, tournament VARCHAR(128), city VARCHAR(128), country VARCHAR(128), neutral VARCHAR(32))")

In [9]:
#Now the table is created. We will fill the entries from csv file. 

for i,row in dataset.iterrows():
            sql = "INSERT INTO football.Results VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
            mycursor.execute(sql, tuple(row))
                        
mydb.commit()

In [10]:
mycursor.execute("Select * from results")

In [11]:
fetch=pd.DataFrame(mycursor.fetchall())
fetch.columns=["Match_date", "Home_Team", "Away_Team", "Home_Score", "Away_score", "Tournament", "City","Country" ,"Neutral"]
fetch

# We fetch this result to double confirm the data import. 

,Match_date,Home_Team,Away_Team,Home_Score,Away_score,Tournament,City,Country,Neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,0
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,0
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,0
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,0
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,0
...,...,...,...,...,...,...,...,...,...
43183,2022-02-01,Suriname,Guyana,2,1,Friendly,Paramaribo,Suriname,0
43184,2022-02-02,Burkina Faso,Senegal,1,3,African Cup of Nations,Yaoundé,Cameroon,1
43185,2022-02-03,Cameroon,Egypt,0,0,African Cup of Nations,Yaoundé,Cameroon,0
43186,2022-02-05,Cameroon,Burkina Faso,3,3,African Cup of Nations,Yaoundé,Cameroon,0


## Total Home-wins by every team

In [12]:
mycursor.execute("Select home_team, count(home_team) from results where home_score>away_score group by home_team")
result1=pd.DataFrame(mycursor.fetchall())
result1.columns=['Team', 'Number of Home Wins']
result1

,Team,Number of Home Wins
0,England,322
1,Scotland,221
2,Wales,129
3,United States,238
4,Northern Ireland,107
...,...,...
284,Matabeleland,1
285,Parishes of Jersey,2
286,South Ossetia,1
287,Chameria,1


## Total away wins by every team

In [13]:

mycursor.execute("Select away_team, count(away_team) as 'Num_away_wins' from results where home_score<away_score group by home_team order by Num_away_wins DESC")
result2=pd.DataFrame(mycursor.fetchall())
result2.columns=['Team', 'Number of Away Wins']
result2

,Team,Number of Away Wins
0,France,170
1,Sweden,161
2,Austria,144
3,England,142
4,Sweden,140
...,...,...
292,Panjab,1
293,Székely Land,1
294,Western Armenia,1
295,Abkhazia,1


## Home wins (non-friendly matches)

In [14]:
mycursor.execute("Select home_team, count(home_team) as Num_home_wins from results where home_score>away_score AND tournament!='Friendly' group by home_team order by Num_home_wins DESC")
result1=pd.DataFrame(mycursor.fetchall())
result1.columns=['Team', 'Number of Home Wins']
result1

,Team,Number of Home Wins
0,Argentina,264
1,Brazil,246
2,South Korea,207
3,England,203
4,Scotland,168
...,...,...
270,Yorkshire,1
271,Parishes of Jersey,1
272,South Ossetia,1
273,Chameria,1


## Away-wins (only non-friendly matches)

In [15]:
mycursor.execute("Select away_team, count(away_team) as Num_away_wins from results where home_score<away_score AND tournament!='Friendly' group by away_team order by Num_away_wins DESC")
result1=pd.DataFrame(mycursor.fetchall())
result1.columns=['Team', 'Number of Away Wins']
result1

,Team,Number of Away Wins
0,England,160
1,South Korea,127
2,Uruguay,126
3,Sweden,114
4,Germany,113
...,...,...
268,Chameria,1
269,Hitra,1
270,Orkney,1
271,São Tomé and Príncipe,1


## Draw Matches (Only non-friendly)

In [16]:
mycursor.execute("Select home_team, count(home_team) as Num_home_wins from results where home_score=away_score AND tournament!='Friendly' group by home_team order by Num_home_wins DESC")
result1=pd.DataFrame(mycursor.fetchall())
result1.columns=['Team', 'Number of draw matches']
result1

,Team,Number of draw matches
0,Argentina,82
1,Malaysia,79
2,South Korea,78
3,Brazil,69
4,Uruguay,68
...,...,...
253,Matabeleland,1
254,Tibet,1
255,Northern Mariana Islands,1
256,Mongolia,1


## Home-wins where margin is greater than or equal to 3 goals (only non-friendly)

In [17]:
mycursor.execute("Select home_team, count(home_team) as Num_home_wins from results where (home_score-away_score)>=3 AND tournament!='Friendly' group by home_team order by Num_home_wins DESC")
result1=pd.DataFrame(mycursor.fetchall())
result1.columns=['Team', 'Number of Home Wins']
result1

,Team,Number of Home Wins
0,Brazil,103
1,Argentina,98
2,England,93
3,South Korea,90
4,Spain,69
...,...,...
248,Cascadia,1
249,Kosovo,1
250,Parishes of Jersey,1
251,Chameria,1


## Away wins where margin is greater than or equal to 3 goals (only non-friendly) 

In [18]:
mycursor.execute("Select away_team, count(away_team) as Num_away_wins from results where (away_score-home_score)>=3 AND tournament!='Friendly' group by away_team order by Num_away_wins DESC")
result1=pd.DataFrame(mycursor.fetchall())
result1.columns=['Team', 'Number of Home Wins']
result1

,Team,Number of Home Wins
0,England,63
1,Uruguay,36
2,Brazil,35
3,Scotland,33
4,Sweden,32
...,...,...
227,United States Virgin Islands,1
228,Chameria,1
229,Turks and Caicos Islands,1
230,Botswana,1


In [19]:
#Now we alter results table and add another column for assigning decades to each observation since we have a date column. 

In [20]:
mycursor.execute('alter table results add Decade INT')

In [21]:
mycursor.execute('select * from results')

In [22]:
pd.DataFrame(mycursor.fetchall())

,0,1,2,3,4,5,6,7,8,9
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,0,None
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,0,None
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,0,None
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,0,None
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,0,None
...,...,...,...,...,...,...,...,...,...,...
43183,2022-02-01,Suriname,Guyana,2,1,Friendly,Paramaribo,Suriname,0,None
43184,2022-02-02,Burkina Faso,Senegal,1,3,African Cup of Nations,Yaoundé,Cameroon,1,None
43185,2022-02-03,Cameroon,Egypt,0,0,African Cup of Nations,Yaoundé,Cameroon,0,None
43186,2022-02-05,Cameroon,Burkina Faso,3,3,African Cup of Nations,Yaoundé,Cameroon,0,None


In [23]:
query=(""" update results
       set Decade= ( 
           CASE 
           
           when match_date >= "1870-1-1" AND match_date < "1880-1-1" then 1870 
           when match_date >= "1880-1-1" AND match_date < "1890-1-1" then 1880 
           when match_date >= "1890-1-1" AND match_date < "1900-1-1" then 1890 
           when match_date >= "1900-1-1" AND match_date < "1910-1-1" then 1900
           when match_date >= "1910-1-1" AND match_date < "1920-1-1" then 1910 
           when match_date >= "1920-1-1" AND match_date < "1930-1-1" then 1920 
           when match_date >= "1930-1-1" AND match_date < "1940-1-1" then 1930 
           when match_date >= "1940-1-1" AND match_date < "1950-1-1" then 1940 
           when match_date >= "1950-1-1" AND match_date < "1960-1-1" then 1950 
           when match_date >= "1960-1-1" AND match_date < "1970-1-1" then 1960 
           when match_date >= "1970-1-1" AND match_date < "1980-1-1" then 1970 
           when match_date >= "1980-1-1" AND match_date < "1990-1-1" then 1980 
           when match_date >= "1990-1-1" AND match_date < "2000-1-1" then 1990 
           when match_date >= "2000-1-1" AND match_date < "2010-1-1" then 2000
           when match_date >= "2010-1-1" AND match_date < "2020-1-1" then 2010
           when match_date >= "2020-1-1" AND match_date < "2030-1-1" then 2020
                    
           else True END);
           
           """)
mydb.commit()
mycursor.execute(query)
mycursor.execute('select * from results')
result10=pd.DataFrame(mycursor.fetchall())


Now we can query few results using decades characteristic. 

## If we were to answer finding the team with the most wins at home for every decade, the query would be that:

In [24]:
query= (""" 
        select * 
        from ( select *,  ROW_NUMBER() OVER(PARTITION BY Decade ORDER BY t.Num_wins DESC)  as abc
        from ( select home_team, Decade, count(home_team) as Num_wins
        from results 
        where home_score>away_score 
        group by Home_team, Decade) t) p
        where p.abc=1
        ORDER BY Decade ASC, p.Num_wins DESC
        
        """)

mycursor.execute(query)

result11=pd.DataFrame(mycursor.fetchall())
result11.columns=['Home_team', 'Decade', 'Num_of_Wins', 'Team_Rank']
result11

# Using this query we can actually find teams with second most home wins for every decade. (Change where condition to p.abc=2) 

,Home_team,Decade,Num_of_Wins,Team_Rank
0,Scotland,1870,5,1
1,Scotland,1880,12,1
2,England,1890,13,1
3,England,1900,10,1
4,Argentina,1910,21,1
5,Argentina,1920,30,1
6,Germany,1930,29,1
7,Argentina,1940,33,1
8,Brazil,1950,44,1
9,Brazil,1960,44,1


## Similarly, we query to find the teams with highest away_wins for every decade

In [25]:
query= (""" 
        select * 
        from ( select *,  ROW_NUMBER() OVER(PARTITION BY Decade ORDER BY t.Num_wins DESC)  as abc
        from ( select away_team, Decade, count(away_team) as Num_wins
        from results 
        where home_score<away_score 
        group by Home_team, Decade) t) p
        where p.abc=1
        ORDER BY Decade ASC, p.Num_wins DESC
        
        """)

mycursor.execute(query)

result11=pd.DataFrame(mycursor.fetchall())
result11.columns=['Home_team', 'Decade', 'Num_of_Away_Wins', 'Team_Rank']
result11

# Using this query we can actually find teams with second most home wins for every decade. (Change where condition to p.abc=2) 

,Home_team,Decade,Num_of_Away_Wins,Team_Rank
0,Scotland,1870,2,1
1,England,1880,9,1
2,England,1890,9,1
3,Scotland,1900,12,1
4,Belgium,1910,8,1
5,Italy,1920,16,1
6,Republic of Ireland,1930,20,1
7,Uruguay,1940,12,1
8,Yugoslavia,1950,22,1
9,Yugoslavia,1960,21,1


## Most drawn matches by a team for every decade

In [26]:
query= (""" 
        select * 
        from ( select *,  ROW_NUMBER() OVER(PARTITION BY Decade ORDER BY t.Num_wins DESC)  as abc
        from ( select home_team, Decade, count(home_team) as Num_wins
        from results 
        where home_score=away_score 
        group by Home_team, Decade) t) p
        where p.abc=1
        ORDER BY Decade ASC, p.Num_wins DESC
        
        """)

mycursor.execute(query)

result11=pd.DataFrame(mycursor.fetchall())
result11.columns=['Team_name', 'Decade', 'Num_of_Draws', 'Team_Rank']
result11

,Team_name,Decade,Num_of_Draws,Team_Rank
0,Scotland,1870,1,1
1,England,1880,2,1
2,Wales,1890,3,1
3,Wales,1900,5,1
4,Argentina,1910,6,1
5,Italy,1920,9,1
6,Germany,1930,12,1
7,Curaçao,1940,7,1
8,Brazil,1950,10,1
9,Malaysia,1960,17,1


In [27]:
mycursor.close()

True